In [1]:
using DataFrames
using Plots
using CSV
using Statistics
using LinearAlgebra

In [2]:
# Makes a new column denoting the class as an integer
function makeClasses(string)
    if string == "Iris-setosa"
        return nothing
    elseif string == "Iris-versicolor"
        return 0
    else
        return 1
    end
end

function sigmoid(z, bias)
    # z equivalent to w_1 * x_1 ...
    # the bias is w_o
    return (1.0 / (1.0 + exp(-(bias[1] + z[1]))))
end

# Computes the gradient over all samples for a given weight using MSEx
function gradient(z, bias, sample, class, n)
    return -(2.0 * sample[1] * exp(-(bias[1] + z[1])) * (class[1] - sigmoid(z, bias))) / (n * (sigmoid(z, bias)[1])^2)
end

gradient (generic function with 1 method)

In [3]:
# Part A
function activation(x)
    if x < 0.5
        return 0
    else return 1
    end
end
# Takes the arguments:
# Data vectors, parameters, and classes, where each vector is a sample
# the first value of the parameter should be the bias
function mean_squared_error(data_vectors, weights, classes)
    meansq = 0
    for i in 1:100
        # Computes the predicted response using the input feature dot the weights and bias term
        predicted = activation(sigmoid(data_vectors[i]' * weights[2:3], weights[1]))
        
        meansq += (classes[i] - predicted[1])^2
    end
    return (meansq/100)
end


mean_squared_error (generic function with 1 method)

In [4]:
# Part E
# Computes the summed gradient for an ensemble of patterns
function sumGradients(dataframe_rows, weights, classes)
    gradients = Array{Float64}(undef, 5)
    gradients[1] = 0
    gradients[2] = 0
    gradients[3] = 0
    # For all of the given data vectors (samples)
    # We sum up the gradient for each respective weight
    for i in 1:size(dataframe_rows)[1]
        data_vector = collect(dataframe_rows[i, :])
        gradients[1] += gradient(data_vector' * weights[2:3], weights[1], [0], classes[i], 100)[1]
        gradients[2] += gradient(data_vector' * weights[2:3], weights[1], data_vector[1], classes[i], 100)[1]
        gradients[3] += gradient(data_vector' * weights[2:3], weights[1], data_vector[2], classes[i], 100)[1]
    end
    return gradients
end

sumGradients (generic function with 1 method)

In [5]:
# Gradient descent
# Calculates the gradient for each weight using the sum across all samples
function gradientDescent(dataframe_rows, weights, classes, stepsize, iterations)
    gradients = sumGradients(dataframe_rows, weights, classes)

    w1 = gradients[1]
    w2 = gradients[2]
    w3 = gradients[3]
    n = 0
    
    w = [w1, w2, w3]
    while (n < iterations)

        #Assigns the new ones
        w1 = w1 - stepsize * gradients[1]
        w2 = w2 - stepsize * gradients[2]
        w3 = w3 - stepsize * gradients[3]
        n += 1
        w = [w1, w2, w3]
        
        gradients = sumGradients(dataframe_rows, w, classes)
    end
    return w
end

gradientDescent (generic function with 1 method)

In [6]:
# Creates the line for the decision boundary
function line(weights)
    y_intercept = -1 * (weights[1] / weights[3])
    slope = -1 * (weights[2] / weights[3])
    return line(x) = slope * x + y_intercept
end

line (generic function with 1 method)

In [ ]:
df = CSV.read("iris.csv", DataFrame)
transform!(df, [:species] => ByRow(makeClasses) => :class)

In [ ]:
# Part B
weights1 = (Float64)[0, 1, 2]
println("Mean Squared error for $weights1 - Large error:")

println(mean_squared_error([collect(row) for row in eachrow(df[51:150, 3:4])], weights1, collect(df[51:150, :class])))

# Finding the decision bound for Large MSE
# For a weight of one mean_squared
irisplot = scatter(df[51:100, 3], df[51:150, 4], color="green", xlabel="Petal Length", ylabel="Petal Width", label="Class 0", title="Iris Set: Neural Networks")
scatter!(irisplot, df[101:150, 3], df[101:150, 4], color="orange", xlabel="Petal Length", ylabel="Petal Width", label="Class 1", title="Iris Set: Neural Networks")
plot!(irisplot, line(weights1), 2.5, 7, label="Large MSE")

In [ ]:
weights2 = [-0.8, -0.03, 0.62]
println("Mean Squared error for weights $weights2 - Small error: ", mean_squared_error([collect(row) for row in eachrow(df[51:150, 3:4])], weights2, collect(df[51:150, :class])))

irisplot = scatter(df[51:100, 3], df[51:150, 4], color="green", xlabel="Petal Length", ylabel="Petal Width", label="Class 0", title="Iris Set: Neural Networks")
scatter!(irisplot, df[101:150, 3], df[101:150, 4], color="orange", xlabel="Petal Length", ylabel="Petal Width", label="Class 1", title="Iris Set: Neural Networks")
plot!(irisplot, line(weights2), 2.5, 7, label="Small MSE")

In [ ]:
# Computes the summed gradient for an ensemble of patterns, with small step size
println("Using a small step size, 0.00001, this is how the boundary changes for one iteration")
classes = collect(df[51:150, :class])
dataframe_rows = df[51:150, 3:4]
weights2 = gradientDescent(dataframe_rows, weights2, classes, 0.0001, 1)

println(weights2)
println("Mean Squared error for weights $weights2 - Small error: ", mean_squared_error([collect(row) for row in eachrow(df[51:150, 3:4])], weights2, collect(df[51:150, :class])))

scatter(df[51:100, 3], df[51:150, 4], color="green", xlabel="Petal Length", ylabel="Petal Width", label="Class 0", title="Iris Set: Neural Networks")
scatter!(df[101:150, 3], df[101:150, 4], color="orange", xlabel="Petal Length", ylabel="Petal Width", label="Class 1", title="Iris Set: Neural Networks")
plot!(line(weights2), 2.5, 7, label="Small MSE")

In [ ]:
println("This is how it changes for several iterations")

weights2 = gradientDescent(dataframe_rows, weights2, classes, 0.0001, 50)

irisplot = scatter(df[51:100, 3], df[51:150, 4], color="green", xlabel="Petal Length", ylabel="Petal Width", label="Class 0", title="Iris Set: Neural Networks")
scatter!(irisplot, df[101:150, 3], df[101:150, 4], color="orange", xlabel="Petal Length", ylabel="Petal Width", label="Class 1", title="Iris Set: Neural Networks")
plot!(irisplot, line(weights2), 2.5, 7, label="Small MSE")

In [ ]:
println("This is how it changes for many iterations")

weights2 = gradientDescent(dataframe_rows, weights2, classes, 0.0001, 99999)

irisplot = scatter(df[51:100, 3], df[51:150, 4], color="green", xlabel="Petal Length", ylabel="Petal Width", label="Class 0", title="Iris Set: Neural Networks")
scatter!(irisplot, df[101:150, 3], df[101:150, 4], color="orange", xlabel="Petal Length", ylabel="Petal Width", label="Class 1", title="Iris Set: Neural Networks")
plot!(irisplot, line(weights2), 2.5, 7, label="Small MSE")